# Prelude

In [1]:
import sys
import pathlib as pl
sys.path.insert(0,str(pl.Path("../").absolute()))

In [2]:

import numpy as np
import pandas as pd
import torch
import torch.nn as nn # neuronal layers, function objects
import torch.nn.functional as F # contains activation functions
from torch.autograd import Variable # wrapper for tensor
import torch.optim as opt
import numpy as np
from utils.data_loader import load_data

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device: {device}")

using device cuda


In [6]:
data_temp, data_wind = load_data()
display(data_temp.head(3))
display(data_wind.head(3))

,timestamp,differential_potential_CH1,differential_potential_CH2,transpiration
0,2022-01-22 13:53:42,526487,467544,442
1,2022-01-22 13:53:44,526542,467315,461
2,2022-01-22 13:53:45,526532,466868,466


,timestamp,differential_potential_CH1,differential_potential_CH2,transpiration
0,2022-01-22 13:53:37,505940,507749,621
1,2022-01-22 13:53:38,505846,507807,632
2,2022-01-22 13:53:40,505990,507764,629


# Experimentation

## Demo Simple Network
For experimentation with pytorch, define a simple network and train data on it

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # dummy parameter to store device
        self.device = device

        # define layers (layers are functions)
        self.lin1 = nn.Linear(10, 10) # 10 features in, 10 features out (== keras dense)
        self.lin2 = nn.Linear(10, 10) # input and output numbers need to match!

    # forward pass data x through network
    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = F.hardtanh(self.lin1(x))
        return x
    

    # calculate how many features are contained in a single data item tensor (num of elements)
    def num_flat_features(self, x):
        size = x.size()[1:]
        num = 1
        for i in size:
            num *= i

        return num

Network Training

In [ ]:
#%%
net = SimpleNet()
net.to(device)
print(net)
#%%
for i in range (100):
    # main training loop / pipeline
    #data = torch.randn(10,10) # 10 data points (directly create batch of 10)
    # define mockup data
    data_x = [1,0,0,0,1,0,0,0,1,1]
    data_y = [0,1,1,1,0,1,1,1,0,0]
    input = Variable(torch.Tensor([data_x for _ in range(10)])).to(net.device) # create as batch (duplicate for demonstration)
    target = Variable(torch.Tensor([data_y for _ in range(10)])).to(net.device) # s.o.
    
    # forward pass
    out = net(input)
    loss = F.mse_loss(out, target)
    net.zero_grad() # net accumulates errors and needs to be reset manually each step

    # backpropagation
    loss.backward()
    learning_rate = 0.1
    optimizer = opt.SGD(net.parameters(), lr=learning_rate) # default stochastic gradient descent
    optimizer.step()
    
    print(loss)
#%%
# example result of trained net and conversion to numpy
x_numpy = np.array([1,0,0,0,1,0,0,0,1,1])
x = torch.Tensor(x_numpy).to(net.device)
y = net(x)
y_numpy = y.detach().cpu().numpy()
print("in  ", x_numpy.astype(np.int8))
print("out ", np.abs(np.rint(y_numpy)).astype(np.int8)) # as nearest (positive) int
# %%